In [1]:
# Imports necesarios
import numpy as np
import pandas as pd
#import seaborn as sb
import matplotlib.pyplot as plt
%matplotlib inline
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
plt.rcParams['figure.figsize'] = (16, 9)
plt.style.use('ggplot')
#from sklearn import linear_model
#from sklearn.metrics import mean_squared_error, r2_score
#from sklearn.neural_network import MLPRegressor

## Abrir Archivos

In [2]:
#cargamos los datos de entrada
data_train = pd.read_csv("/Users/manuelmoya/Documents/USACH/2019-1er/Tesis/1.Desarrollo/05.AnalisisDatos/Datos/train.csv",header=0)
data_wf1 = pd.read_csv("/Users/manuelmoya/Documents/USACH/2019-1er/Tesis/1.Desarrollo/05.AnalisisDatos/Datos/wf1.csv",header=0)
data_bench = pd.read_csv("/Users/manuelmoya/Documents/USACH/2019-1er/Tesis/1.Desarrollo/05.AnalisisDatos/Datos/benchmark.csv",header=0)

#veamos cuantas dimensiones y registros contiene
#data_train.shape


In [3]:
# Ver los 1ros registros
data_train.head()

,date,wp1,wp2,wp3,wp4,wp5,wp6,wp7
0,2009070100,0.045,0.233,0.494,0.105,0.056,0.118,0.051
1,2009070101,0.085,0.249,0.257,0.105,0.066,0.066,0.051
2,2009070102,0.020,0.175,0.178,0.033,0.015,0.026,0.000
3,2009070103,0.060,0.085,0.109,0.022,0.010,0.013,0.000
4,2009070104,0.045,0.032,0.079,0.039,0.010,0.000,0.000


In [4]:
data_wf1.head()

,date,hors,u,v,ws,wd
0,2009070100,1,2.34,-0.79,2.47,108.68
1,2009070100,2,2.18,-0.99,2.40,114.31
2,2009070100,3,2.20,-1.21,2.51,118.71
3,2009070100,4,2.35,-1.40,2.73,120.86
4,2009070100,5,2.53,-1.47,2.93,120.13


In [5]:
data_bench.head()

,id,date,wp1,wp2,wp3,wp4,wp5,wp6,wp7
0,1,2011010101,0.551,0.0,0.968,0.645,0.9,0.921,0.833
1,2,2011010102,0.551,0.0,0.968,0.645,0.9,0.921,0.833
2,3,2011010103,0.551,0.0,0.968,0.645,0.9,0.921,0.833
3,4,2011010104,0.551,0.0,0.968,0.645,0.9,0.921,0.833
4,5,2011010105,0.551,0.0,0.968,0.645,0.9,0.921,0.833


In [6]:
base_data = data_train.append(data_bench)

In [7]:
base_data

,date,wp1,wp2,wp3,wp4,wp5,wp6,wp7,id
0,2009070100,0.045,0.233,0.494,0.105,0.056,0.118,0.051,NaN
1,2009070101,0.085,0.249,0.257,0.105,0.066,0.066,0.051,NaN
2,2009070102,0.020,0.175,0.178,0.033,0.015,0.026,0.000,NaN
3,2009070103,0.060,0.085,0.109,0.022,0.010,0.013,0.000,NaN
4,2009070104,0.045,0.032,0.079,0.039,0.010,0.000,0.000,NaN
...,...,...,...,...,...,...,...,...,...
7483,2012062808,0.226,0.069,0.227,0.011,0.329,0.079,0.000,7484.0
7484,2012062809,0.226,0.069,0.227,0.011,0.329,0.079,0.000,7485.0
7485,2012062810,0.226,0.069,0.227,0.011,0.329,0.079,0.000,7486.0
7486,2012062811,0.226,0.069,0.227,0.011,0.329,0.079,0.000,7487.0


## Agregar campos auxiliares a wf1

In [10]:
# agregar anno
def anno(num):
    num_str = str(num)
    num_corto = num_str[0:4]
    return int(num_corto)

# agregar mes
def mes(num):
    num_str = str(num)
    num_corto = num_str[4:6]
    return int(num_corto)

# agregar mes
def dia(num):
    num_str = str(num)
    num_corto = num_str[6:8]
    return int(num_corto)

# agregar hrsObs
def horsObs(num):
    num_str = str(num)
    num_corto = num_str[8:10]
    return int(num_corto)

def upd_date(num):
    num_str = str(num)
    num_corto = num_str[0:8]
    return int(num_corto)


# Crear campos
data_wf1['anno'] = data_wf1['date'].apply(anno)
data_wf1['mes'] = data_wf1['date'].apply(mes)
data_wf1['dia'] = data_wf1['date'].apply(dia)
data_wf1['horsObs'] = data_wf1['date'].apply(horsObs)

# Actualizar date
data_wf1['date'] = data_wf1['date'].apply(upd_date)

data_wf1['daysAdd'] = -5
data_wf1['horsFore']= -5
data_wf1['dateFore']= -5
data_wf1['wp1']= -5

data_wf1

,date,hors,u,v,ws,wd,anno,mes,dia,horsObs,daysAdd,horsFore,dateFore,wp1
0,20090701,1,2.34,-0.79,2.47,108.68,2009,7,1,0,-5,-5,-5,-5
1,20090701,2,2.18,-0.99,2.40,114.31,2009,7,1,0,-5,-5,-5,-5
2,20090701,3,2.20,-1.21,2.51,118.71,2009,7,1,0,-5,-5,-5,-5
3,20090701,4,2.35,-1.40,2.73,120.86,2009,7,1,0,-5,-5,-5,-5
4,20090701,5,2.53,-1.47,2.93,120.13,2009,7,1,0,-5,-5,-5,-5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104827,20120626,44,2.44,1.08,2.67,66.14,2012,6,26,12,-5,-5,-5,-5
104828,20120626,45,2.11,1.46,2.57,55.39,2012,6,26,12,-5,-5,-5,-5
104829,20120626,46,1.63,1.61,2.29,45.42,2012,6,26,12,-5,-5,-5,-5
104830,20120626,47,1.07,1.58,1.91,34.05,2012,6,26,12,-5,-5,-5,-5


## Actualizar campos auxiliares

In [11]:
# daysAdd
def asigna_daysAdd(hors, horsObs):
    suma=0
    if horsObs == 0:
        if hors > 24:
           suma=1           
    #else:
    #    if (hors > 12) & (hors <= 36):
    #      suma=1
    #    elif hors > 36: 
    #      suma=2
    return suma

data_wf1['daysAdd'] = data_wf1.apply(lambda x: asigna_daysAdd(x['hors'],x['horsObs']), axis=1);

In [12]:
# horsFore
def asigna_horsFore(hors, horsObs):
    hrs=hors
    if horsObs == 0:
        if hors > 24:
           hrs=hors-24         
    #else:
    #    if (hors > 12) & (hors <= 36):
    #      hrs=hors-12 
    #    elif hors > 36: 
    #      hrs=hors-36
        
    if hrs==24:
        hrs=00
        
    return int(hrs)

data_wf1['horsFore'] = data_wf1.apply(lambda x: asigna_horsFore(x['hors'],x['horsObs']), axis=1);

In [13]:
#data_wf1.head(50)

In [16]:
# dateFore
from datetime import timedelta,datetime

# Convertir num 2 digitos con cero a la izquierda
def conv_num_2D(num):
  num_conv = "0" + str(num)
  num_final = num_conv[len(num_conv)-2:len(num_conv)]
  
  return num_final


# Asignar dateFore
def asigna_dateFore(anno, mes, dia,daysAdd, horsFore):
    fecha = str(int(anno)) +"-"+ str(int(mes)) +"-"+ str(int(dia))
    datetime_obj = datetime.strptime(fecha, '%Y-%m-%d')
    fecha_fin = datetime_obj + timedelta(days=daysAdd)
    #date_time = datetime.fromtimestamp(fecha_fin)
    fecha_format= fecha_fin.strftime("%Y%m%d")
    fecha_format = fecha_format + conv_num_2D(horsFore)
    
    return int(fecha_format)

data_wf1['dateFore'] = data_wf1.apply(lambda x: asigna_dateFore(x['anno'],x['mes'],x['dia'],x['daysAdd'],x['horsFore']), axis=1);
data_wf1

,date,hors,u,v,ws,wd,anno,mes,dia,horsObs,daysAdd,horsFore,dateFore,wp1
0,20090701,1,2.34,-0.79,2.47,108.68,2009,7,1,0,0,1,2009070101,-5
1,20090701,2,2.18,-0.99,2.40,114.31,2009,7,1,0,0,2,2009070102,-5
2,20090701,3,2.20,-1.21,2.51,118.71,2009,7,1,0,0,3,2009070103,-5
3,20090701,4,2.35,-1.40,2.73,120.86,2009,7,1,0,0,4,2009070104,-5
4,20090701,5,2.53,-1.47,2.93,120.13,2009,7,1,0,0,5,2009070105,-5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104827,20120626,44,2.44,1.08,2.67,66.14,2012,6,26,12,0,44,2012062644,-5
104828,20120626,45,2.11,1.46,2.57,55.39,2012,6,26,12,0,45,2012062645,-5
104829,20120626,46,1.63,1.61,2.29,45.42,2012,6,26,12,0,46,2012062646,-5
104830,20120626,47,1.07,1.58,1.91,34.05,2012,6,26,12,0,47,2012062647,-5


In [17]:
#temp_ = data_wf1[(data_wf1['date'] ==  20090701) & (data_wf1['horsObs'] ==  12)]
#temp_

In [26]:
# Asignar wp1
def asigna_wp1(dateFore):
    busq_wp1 = base_data['wp1'][ (base_data['date'] ==  dateFore)]
    
    #val_final= pd.NA
    val_final= None
    if len(busq_wp1)>0:
         val_final=busq_wp1.values[0]
    
    return val_final

data_wf1['wp1'] = data_wf1.apply(lambda x: asigna_wp1(x['dateFore']), axis=1);

In [27]:
data_wf1

,date,hors,u,v,ws,wd,anno,mes,dia,horsObs,daysAdd,horsFore,dateFore,wp1
0,20090701,1,2.34,-0.79,2.47,108.68,2009,7,1,0,0,1,2009070101,0.085
1,20090701,2,2.18,-0.99,2.40,114.31,2009,7,1,0,0,2,2009070102,0.020
2,20090701,3,2.20,-1.21,2.51,118.71,2009,7,1,0,0,3,2009070103,0.060
3,20090701,4,2.35,-1.40,2.73,120.86,2009,7,1,0,0,4,2009070104,0.045
4,20090701,5,2.53,-1.47,2.93,120.13,2009,7,1,0,0,5,2009070105,0.035
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104827,20120626,44,2.44,1.08,2.67,66.14,2012,6,26,12,0,44,2012062644,NaN
104828,20120626,45,2.11,1.46,2.57,55.39,2012,6,26,12,0,45,2012062645,NaN
104829,20120626,46,1.63,1.61,2.29,45.42,2012,6,26,12,0,46,2012062646,NaN
104830,20120626,47,1.07,1.58,1.91,34.05,2012,6,26,12,0,47,2012062647,NaN


In [28]:
#backup_id = df.id
output = data_wf1.apply(pd.Series.interpolate)
output

#output['wp1'].fillna((pd.Series.interpolate), inplace=True)
#output

#output.id = backup_id

,date,hors,u,v,ws,wd,anno,mes,dia,horsObs,daysAdd,horsFore,dateFore,wp1
0,20090701,1,2.34,-0.79,2.47,108.68,2009,7,1,0,0,1,2009070101,0.085
1,20090701,2,2.18,-0.99,2.40,114.31,2009,7,1,0,0,2,2009070102,0.020
2,20090701,3,2.20,-1.21,2.51,118.71,2009,7,1,0,0,3,2009070103,0.060
3,20090701,4,2.35,-1.40,2.73,120.86,2009,7,1,0,0,4,2009070104,0.045
4,20090701,5,2.53,-1.47,2.93,120.13,2009,7,1,0,0,5,2009070105,0.035
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104827,20120626,44,2.44,1.08,2.67,66.14,2012,6,26,12,0,44,2012062644,0.556
104828,20120626,45,2.11,1.46,2.57,55.39,2012,6,26,12,0,45,2012062645,0.556
104829,20120626,46,1.63,1.61,2.29,45.42,2012,6,26,12,0,46,2012062646,0.556
104830,20120626,47,1.07,1.58,1.91,34.05,2012,6,26,12,0,47,2012062647,0.556


## Deja solo los datos con valores

## Eliminar pronosticos menor a 5 horas

In [29]:
temp_ = output[(output['hors'] >=  5)]
data_test_fin = temp_

data_test_fin
#frm_final.corr()

,date,hors,u,v,ws,wd,anno,mes,dia,horsObs,daysAdd,horsFore,dateFore,wp1
4,20090701,5,2.53,-1.47,2.93,120.13,2009,7,1,0,0,5,2009070105,0.035
5,20090701,6,2.66,-1.29,2.96,115.79,2009,7,1,0,0,6,2009070106,0.005
6,20090701,7,2.69,-0.81,2.81,106.71,2009,7,1,0,0,7,2009070107,0.000
7,20090701,8,2.72,-0.26,2.73,95.39,2009,7,1,0,0,8,2009070108,0.000
8,20090701,9,2.87,0.08,2.87,88.50,2009,7,1,0,0,9,2009070109,0.010
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104827,20120626,44,2.44,1.08,2.67,66.14,2012,6,26,12,0,44,2012062644,0.556
104828,20120626,45,2.11,1.46,2.57,55.39,2012,6,26,12,0,45,2012062645,0.556
104829,20120626,46,1.63,1.61,2.29,45.42,2012,6,26,12,0,46,2012062646,0.556
104830,20120626,47,1.07,1.58,1.91,34.05,2012,6,26,12,0,47,2012062647,0.556


In [30]:
#data_test_fin.corr()

## Borrar campos auxiliares

In [31]:
data_test_fin.drop(['date','u','v','daysAdd','horsFore','dateFore'], axis=1, inplace=True)
data_test_fin

/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,hors,ws,wd,anno,mes,dia,horsObs,wp1
4,5,2.93,120.13,2009,7,1,0,0.035
5,6,2.96,115.79,2009,7,1,0,0.005
6,7,2.81,106.71,2009,7,1,0,0.000
7,8,2.73,95.39,2009,7,1,0,0.000
8,9,2.87,88.50,2009,7,1,0,0.010
...,...,...,...,...,...,...,...,...
104827,44,2.67,66.14,2012,6,26,12,0.556
104828,45,2.57,55.39,2012,6,26,12,0.556
104829,46,2.29,45.42,2012,6,26,12,0.556
104830,47,1.91,34.05,2012,6,26,12,0.556


In [32]:
#data_test_fin.wp1.round(3)


def round_dec(num):    
    return round(num,3)

# Redondear wp1
data_test_fin['wp1'] = data_test_fin['wp1'].apply(round_dec)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [33]:
#temp2 = data_test_fin.dropna() 
#data_test_fin = temp2

In [34]:
data_test_fin

,hors,ws,wd,anno,mes,dia,horsObs,wp1
4,5,2.93,120.13,2009,7,1,0,0.035
5,6,2.96,115.79,2009,7,1,0,0.005
6,7,2.81,106.71,2009,7,1,0,0.000
7,8,2.73,95.39,2009,7,1,0,0.000
8,9,2.87,88.50,2009,7,1,0,0.010
...,...,...,...,...,...,...,...,...
104827,44,2.67,66.14,2012,6,26,12,0.556
104828,45,2.57,55.39,2012,6,26,12,0.556
104829,46,2.29,45.42,2012,6,26,12,0.556
104830,47,1.91,34.05,2012,6,26,12,0.556


## Grabar datos finales

In [35]:

data_test_fin.to_csv('/Users/manuelmoya/Documents/USACH/2019-1er/Tesis/1.Desarrollo/05.AnalisisDatos/Datos/wf1_train_fin_mod3.csv')
